# Модель предсказания типа лога (LinearSVC)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import joblib
import csv

In [ ]:
# Необхоимо импортировать файл audit.log
from google.colab import files
uploaded = files.upload()

Saving audit.log to audit.log


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Подготовка данных

In [ ]:
# Словарь переводов по типу
log_types_translations = {
    "ADD_USER": "Добавление пользователя",
    "CWD": "Текущий рабочий каталог",
    "USER_AVC": "Пользовательский контроль доступа (Access Vector Cache)",
    "USER_AUTH": "Аутентификация пользователя",
    "USER_START": "Начало сессии пользователя",
    "CRED_REFR": "Обновление учетных данных",
    "USER_CMD": "Команда пользователя",
    "EXECVE": "Выполнение программы",
    "CONFIG_CHANGE": "Изменение конфигурации",
    "PATH": "Путь",
    "SYSCALL": "Системный вызов",
    "SERVICE_STOP": "Остановка службы",
    "USER_END": "Завершение сессии пользователя",
    "BPF": "Berkeley Packet Filter",
    "CRED_DISP": "Удаление учетных данных",
    "CRED_ACQ": "Приобретение учетных данных",
    "SOCKADDR": "Сетевой адрес",
    "LOGIN": "Вход в систему",
    "AVC": "Контроль доступа (Access Vector Cache)",
    "DAEMON_END": "Завершение демона",
    "PROCTITLE": "Название процесса",
    "SYSTEM_RUNLEVEL": "Уровень выполнения системы",
    "SERVICE_START": "Запуск службы",
    "BPRM_FCAPS": "Капсулы файловой системы",
    "USER_ACCT": "Учетная запись пользователя",
    "SYSTEM_BOOT": "Загрузка системы",
    "DAEMON_START": "Запуск демона",
    "ANOM_ABEND": "Аномальное завершение"
}

In [ ]:
# Функция для извлечения типа из строки лога
def extract_event_type(line):
    parts = line.split()
    for part in parts:
        if part.startswith("type="):
            return part.split("=")[1]
    return None

# Открытие файлов для чтения и записи
with open("audit.log", "r") as log_file, open("audit_svc_data.csv", "w", newline="", encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["log_message", "event_type"])

    for line in log_file:
        event_type = extract_event_type(line)
        if event_type:
            translated_event_type = log_types_translations.get(event_type, event_type)
            csv_writer.writerow([line.strip(), translated_event_type])


In [ ]:
# Наполнение датасета
data = pd.read_csv("audit_svc_data.csv", nrows=1000)
data.head()

,log_message,event_type
0,type=DAEMON_START msg=audit(1714314919.366:346...,Запуск демона
1,type=SYSCALL msg=audit(1714314919.363:537): ar...,Системный вызов
2,type=SOCKADDR msg=audit(1714314919.363:537): s...,Сетевой адрес
3,type=PROCTITLE msg=audit(1714314919.363:537): ...,Название процесса
4,type=CONFIG_CHANGE msg=audit(1714314919.363:53...,Изменение конфигурации


In [ ]:
# Информация о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   log_message  1000 non-null   object
 1   event_type   1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [ ]:
# Подготовка данных
X = data['log_message']
y = data['event_type']

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Train dataset size: {X_train.shape[0]}, {y_train.shape[0]}')
print(f'Test dataset size: {X_test.shape[0]}, {y_test.shape[0]}')

Train dataset size: 800, 800
Test dataset size: 200, 200


### Обучение модели

In [ ]:
# Векторизация текста (логов)
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели
svm_model = LinearSVC()
svm_model.fit(X_train_tfidf, y_train)

# Предсказание на тестовом наборе
y_pred = svm_model.predict(X_test_tfidf)

# Оценка производительности модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Вывод отчета о классификации
print(classification_report(y_test, y_pred))

# Экспорт модели
joblib.dump(svm_model, 'svm_model.joblib')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

Accuracy: 0.99
                                                         precision    recall  f1-score   support

                                 Berkeley Packet Filter       1.00      1.00      1.00        34
                            Аутентификация пользователя       1.00      1.00      1.00         3
                                         Вход в систему       1.00      1.00      1.00         5
                         Завершение сессии пользователя       1.00      1.00      1.00         2
                                       Загрузка системы       0.00      0.00      0.00         2
                                          Запуск службы       0.93      1.00      0.97        28
                                 Изменение конфигурации       1.00      1.00      1.00         3
                                   Команда пользователя       1.00      1.00      1.00         2
                 Контроль доступа (Access Vector Cache)       1.00      1.00      1.00        15
              

D:\Program Files\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['tfidf_vectorizer.joblib']

### Тестирование

In [ ]:
# Импорт модели
loaded_svm_model = joblib.load('svm_model.joblib')
loaded_tfidf_vectorizer = joblib.load('tfidf_vectorizer.joblib')

# Тестовые логи
new_logs = [ "type=SYSCALL msg=audit(1714314919.363:537): arch=c000003e syscall=44 success=yes exit=60 a0=3 a1=7ffe69e55e30 a2=3c a3=0 items=0 ppid=5712 pid=5713 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=4294967295 comm='auditd' exe='/usr/sbin/auditd' subj=unconfined key=(null)ARCH=x86_64 SYSCALL=sendto AUID='unset' UID='root' GID='root' EUID='root' SUID='root' FSUID='root' EGID='root' SGID='root' FSGID='root'",
            "type=CWD msg=audit(1714314919.419:539): cwd='/'",
             "type=PROCTITLE msg=audit(1714314919.363:538): proctitle='/sbin/auditd'" ]

# Векторизация логов
log_vectors = []
for log in new_logs:
  log_vectors.append(loaded_tfidf_vectorizer.transform([log]))

# Предсказание событий на основе новых логов
event_preds = []
for vec in log_vectors:
  event_preds.append(loaded_svm_model.predict(vec))

# Вывод предсказанных событий
for log, pred in zip(new_logs, event_preds):
  print(f"Event: {pred[0]}\nLog: {log}\n")


Event: Системный вызов
Log: type=SYSCALL msg=audit(1714314919.363:537): arch=c000003e syscall=44 success=yes exit=60 a0=3 a1=7ffe69e55e30 a2=3c a3=0 items=0 ppid=5712 pid=5713 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=4294967295 comm='auditd' exe='/usr/sbin/auditd' subj=unconfined key=(null)ARCH=x86_64 SYSCALL=sendto AUID='unset' UID='root' GID='root' EUID='root' SUID='root' FSUID='root' EGID='root' SGID='root' FSGID='root'

Event: Текущий рабочий каталог
Log: type=CWD msg=audit(1714314919.419:539): cwd='/'

Event: Название процесса
Log: type=PROCTITLE msg=audit(1714314919.363:538): proctitle='/sbin/auditd'

